## Dropout!
### In this notebook
#### 1. Verifying pytorch dropout is applied only during inference, scaling of activations
#### 2. Simplistic Implementation of dropout

#### During training, randomly zeroes some of the elements of the input tensor with probability p
#### During inference, dropout is not applied
#### dropout output is scaled at training or inference/test (inverted dropout)

In [1]:
import torch
import torch.nn as nn

#### Verifying pytorch dropout is applied only during inference, scaling of activations

In [2]:
class JustDropoutModel(nn.Module):
    def __init__(self, dropout_prob):
        super(JustDropoutModel, self).__init__()
        self.dropout = nn.Dropout(p=dropout_prob,inplace=False) # inplace is set to False so that input itself is not updated

    def forward(self, x):
        x = self.dropout(x)  # Apply dropout during training
        return x

In [3]:
model = JustDropoutModel(0.5)
model.train() # Mode of model to training

JustDropoutModel(
  (dropout): Dropout(p=0.5, inplace=False)
)

In [4]:
X = torch.randn(2, 2)
X

tensor([[ 0.1984, -1.0945],
        [-0.2061,  1.8554]])

In [5]:
model(X)

tensor([[0.3968, -0.0000],
        [-0.0000, 0.0000]])

In [8]:
# Randomly made some input to zero
# Scaled the other outputs by 1/(1-p) Ex: 0.1984/(1-0.5)=0.3968
# Scaling during training time as it is a inverted dropout

In [6]:
#putting model into inference mode
model.eval()

JustDropoutModel(
  (dropout): Dropout(p=0.5, inplace=False)
)

In [7]:
model(X)
# no Change during inference phase

tensor([[ 0.1984, -1.0945],
        [-0.2061,  1.8554]])

#### Simplistic Implementation of dropout

In [15]:
class JustDropoutModelImpl(nn.Module):
    def __init__(self, dropout_prob):
        super(JustDropoutModelImpl, self).__init__()
        self.dropout_prob=dropout_prob
       
    def forward(self, x):
        x = self.dropout(x,self.dropout_prob)
        return x
    
    
    def dropout(self, x, dropout_prob):
        if self.training:  # dropout only for training, return as is for inference
            mask = (torch.rand(x.shape) > dropout_prob).float()
            x = x * mask / (1.0 - dropout_prob) # xcale the activations like pytorch droput does
        return x

In [21]:
#repeating what we did for pytorch dropout

In [16]:
model = JustDropoutModelImpl(0.5)
model.train() # Mode of model to training

JustDropoutModelImpl()

In [17]:
X

tensor([[ 0.1984, -1.0945],
        [-0.2061,  1.8554]])

In [18]:
model(X)

tensor([[ 0.3968, -0.0000],
        [-0.4121,  3.7108]])

In [19]:
model.eval()

JustDropoutModelImpl()

In [20]:
model(X)

tensor([[ 0.1984, -1.0945],
        [-0.2061,  1.8554]])